In [1]:
import gym
import jax
import tax
import tree
import tqdm
import optax
import jax.numpy as jnp
import numpy as np
import haiku as hk

from a2c.a2c import Data
from a2c.a2c import State
from a2c.a2c import Batch
from a2c.a2c import process_data
from a2c.a2c import loss_fn
from a2c.common.utils import evaluation
from a2c.common.nn import mlp_multivariate_normal_diag
from a2c.common.nn import mlp_deterministic
from jax import jit
from jax import vmap
from gym.vector import AsyncVectorEnv
from functools import partial

tax.set_platform('cpu')
rng = jax.random.PRNGKey(42)

In [2]:
def init_nn(
    observation_size, action_size, seed=42,
    policy_opt = 'rmsprop',
    policy_opt_kwargs = dict(learning_rate=1e-3),
    value_opt = 'rmsprop',
    value_opt_kwargs = dict(learning_rate=1e-3),
    policy_kwargs: dict = dict(), value_kwargs: dict = dict(),
):
    rng = jax.random.PRNGKey(seed)
    dummy_action = jnp.zeros((action_size))
    dummy_observation = jnp.zeros((observation_size))
    
    policy_def = lambda x: mlp_multivariate_normal_diag(action_size, **policy_kwargs)(x)
    policy_def = hk.transform(policy_def)
    policy_def = hk.without_apply_rng(policy_def)
    policy_opt = getattr(optax, policy_opt)(**policy_opt_kwargs)
    
    value_def = lambda x: mlp_deterministic(1, **value_kwargs)(x).squeeze(-1)
    value_def = hk.transform(value_def)
    value_def = hk.without_apply_rng(value_def)
    value_opt = getattr(optax, value_opt)(**value_opt_kwargs)
    
    rng, rng_policy, rng_value = jax.random.split(rng, 3)
    value_params = value_def.init(rng_policy, dummy_observation)
    value_opt_state = value_opt.init(value_params)
    policy_params = policy_def.init(rng_policy, dummy_observation)
    policy_opt_state = policy_opt.init(policy_params)

    params = {'policy': policy_params, 'value': value_params}
    opt_state = {'policy': policy_opt_state, 'value': value_opt_state}
    
    process_data_to_batch = partial(
        process_data, value_apply=value_def.apply,    
    )
    
    loss = partial(loss_fn, value_apply=value_def.apply, policy_apply=policy_def.apply)

    @jit
    def update_fn(state, inputs):
        """ Generic Update function """
        g, metrics = jax.grad(loss, has_aux=True)(state.params, inputs)

        updates, value_opt_state = value_opt.update(g['value'], state.opt_state['value'])
        value_params = jax.tree_multimap(lambda p, u: p + u, state.params['value'], updates)

        updates, policy_opt_state = policy_opt.update(g['policy'], state.opt_state['policy'])
        policy_params = jax.tree_multimap(lambda p, u: p + u, state.params['policy'], updates)

        params = state.params
        params = dict(policy=policy_params, value=value_params)
        opt_state = state.opt_state
        opt_state = dict(policy=policy_opt_state, value=value_opt_state)
        state = state.replace(params=params, opt_state=opt_state)
        return state, metrics


    def interaction(env, horizon: int = 10, seed: int = 42):
        rng = jax.random.PRNGKey(seed)
        observation, buf = env.reset(), []
        params = yield

        while True:
            for _ in range(horizon):
                rng, rng_action = jax.random.split(rng)
                action = jit(policy_def.apply)(params, observation).sample(seed=rng_action)
                action = np.array(action)
                observation_next, reward, done, info = env.step(action)
                buf.append({
                    'observation': observation,
                    'reward': reward,
                    'terminal': 1. - done,
                    'action': action
                })
                observation = observation_next.copy()

            data = jit(tax.reduce)(buf)
            data['last_observation'] = observation
            params = yield data
            buf = []
    
    
    def make_policy(params):
        fn = lambda rng, x: policy_def.apply(params, x).sample(seed=rng) 
        return jit(fn)
    
    return State(key=rng, params=params, opt_state=opt_state), {
        'interaction': interaction,
        'process_data': jit(process_data_to_batch),
        'make_policy': make_policy,
        'loss': jit(loss), 'update': jit(update_fn),
        'full_update': jit(lambda state, data: update_fn(state, process_data_to_batch(state.params, data)))
    }

In [3]:
NEnvs = 8
make_env = lambda: gym.make('Pendulum-v0')

env = AsyncVectorEnv([make_env for _ in range(NEnvs)])
env_test = gym.make('Pendulum-v0')
action_size = env_test.action_space.shape[0]
observation_size = env_test.observation_space.shape[0]
state, a2c = init_nn(observation_size, action_size, 42, policy_kwargs={}, value_kwargs={})

In [4]:
interaction_step = a2c['interaction'](env, 10)
interaction_step.send(None)

In [ ]:
while True:
    for _ in tqdm.notebook.trange(100):
        data = interaction_step.send(state.params['policy'])
        state, info = a2c['full_update'](state, data)
    info_eval = evaluation(rng, env_test, a2c['make_policy'](state.params['policy']))
    print(info_eval)

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval/score': -1597.8486729371248, 'eval/score_std': 189.7525478272289}


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval/score': -1659.1009802747253, 'eval/score_std': 146.6306508327761}


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval/score': -1503.6984855376463, 'eval/score_std': 13.425677816131966}


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval/score': -1474.4637557825886, 'eval/score_std': 23.530218250238455}


  0%|          | 0/100 [00:00<?, ?it/s]